## Deep Neural Networks with Keras

Barzon Giacomo 1207274 <br>
Kabbur Hanumanthappa Manjunatha Karan 1236383 <br>
Rugel Wolfgang 1218362 <br>

### 1. Data augmentation

In [ ]:
import math
import csv
import numpy as np

# reproducibility
np.random.seed(12345)

# digits 1....9
D = 9

# loading the database
fname = 'secretkeys_exe.csv'
dataset = np.loadtxt(fname, delimiter=",", dtype=int) 
N = len(dataset) 
print(f'data: {N}')

# extract data and labels
s = dataset[:,0]
y = dataset[:,-1]

In [ ]:
# Augmenting the data by including all the L-1 cyclic combinations of the data.
def augment(dataset):
    s = dataset[:,0]
    y = dataset[:,-1]
    L = len(str(s[0]))
    augmented_list = []
    for i in range(len(s)):
        augmented_list.append([s[i],y[i]])
        string = str(s[i])
        for j in range(L-1):
            rotate = int(string[-1]+string[:-1])
            string = str(rotate)
            augmented_list.append([rotate,y[i]])
    return augmented_list

augmented_array = np.array(augment(dataset))
print(augmented_array[:10])
    
# Randomly permute the data
augmented_array = np.random.permutation(augmented_array)
augmented_array = np.random.permutation(augmented_array)
print(augmented_array[:10])

# Save .txt file
np.savetxt('Randomized_Augmented_secretkeys.csv',augmented_array,delimiter = ',',fmt='%d')

### 2. Grid-search

In [ ]:
import matplotlib.pyplot as plt

from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.regularizers import l2
from keras.optimizers import SGD, RMSprop, Adam
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.utils import to_categorical
from keras.models import load_model

In [ ]:
# Define a DNN class
# input: grid-search parameters
# output: .png with train/test curves, .h5 with best weights, best model test/train accuracy 

class myDNN():
    
    def __init__(self, params):
        self.D = 9 # digits 1...9
        
        self.activation = params[0]
        self.optimizer = params[1]
        self.lrate = params[2]
        self.batch_size = params[3]
        self.lreg = params[4]
        self.winit = params[5]
        self.dropout = params[6]
        
        self.loadData()
        self.initNetwork()
        
        
    def loadData(self):
        
        # Load csv dataset
        data = 'Randomized_Augmented_secretkeys.csv'
        data = np.loadtxt(data, delimiter=',',dtype=int)
        
        # Extract labels
        labels = data[:,-1]
        
        # Extract samples and perform one-hot encoding
        samples = data[:,0].astype(str)
        self.L = len(str(samples[0])) # number of digits for eachs string
        samples = [int(i) for j in range(samples.shape[0]) for i in samples[j]] # expand string
        samples = np.array(samples).reshape(-1,self.L) -1
        samples = to_categorical(samples).reshape(-1,self.L*self.D) # one-hot encoding
        
        # Split samples into train and test set
        Ntrain = int(0.8 * labels.shape[0])
        self.x_train, self.x_test = samples[:Ntrain], samples[Ntrain:]
        self.y_train, self.y_test = labels[:Ntrain], labels[Ntrain:]
        
        
    def initNetwork(self):
        inputDim = self.L * self.D # samples shape
        
        # define Keras model
        self.model = Sequential()
        self.model.add(Dense(inputDim, input_shape=(inputDim,), activation=self.activation, 
                             kernel_regularizer=l2(self.lreg), bias_regularizer=l2(self.lreg),
                            kernel_initializer=self.winit))
        self.model.add(Dropout(self.dropout))
        self.model.add(Dense(max(10,int(inputDim/2)), activation=self.activation, 
                             kernel_regularizer=l2(self.lreg), bias_regularizer=l2(self.lreg),
                            kernel_initializer=self.winit))
        self.model.add(Dropout(self.dropout))
        self.model.add(Dense(max(6,int(inputDim/4)), activation=self.activation, 
                             kernel_regularizer=l2(self.lreg), bias_regularizer=l2(self.lreg),
                            kernel_initializer=self.winit))
        self.model.add(Dropout(self.dropout))
        self.model.add(Dense(1, activation='sigmoid'))
        #print(self.model.summary())
        
        # define optimizer
        if self.optimizer == 'SGD':
            opt = SGD(lr=self.lrate, decay=1e-6, momentum=0.7, nesterov=True)
        elif self.optimizer == 'RMSprop':
            opt = RMSprop(learning_rate=self.lrate)
        elif self.optimizer == 'Adam':
            opt = Adam(learning_rate=self.lrate)
        
        # compile model
        self.model.compile(loss='binary_crossentropy',
                           optimizer=opt,
                           metrics=['accuracy'])
        
        
    def train(self, printGraphs=True, verbose=False):
        print('START TRAINING MODEL')
        
        # filename for saving best model
        filename = 'a' + str(self.activation) + '_o' + str(self.optimizer) + '_l' + str(self.lrate)
        filename += '_b' + str(self.batch_size) +'_r' + str(self.lreg) + '_w' + str(self.winit)
        filename += '_d' + str(self.dropout)
        weights_file = filename + '.h5'
        
        # implement early-stopping
        es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=10)
        
        # implement checkpoint for returning the best model
        mc = ModelCheckpoint(weights_file,
                             monitor='val_accuracy', mode='max', verbose=0, save_best_only=True)
        
        # train model
        fit = self.model.fit(self.x_train, self.y_train, validation_data=(self.x_test, self.y_test),
                            epochs = 200, batch_size = self.batch_size, callbacks=[es, mc],
                            verbose=verbose)
        
        # load the best model
        self.model = load_model(weights_file)
        
        # evaluate the model
        _, self.train_acc = self.model.evaluate(self.x_train, self.y_train, verbose=0)
        _, self.test_acc = self.model.evaluate(self.x_test, self.y_test, verbose=0)
        print('Train accuracy: %.3f, Test accuracy: %.3f' % (self.train_acc, self.test_acc))
        
        # save learning curves
        graph_file = filename + '.png'
        
        fig = plt.figure(figsize=(15,6))
        
        # accuracy
        plt.subplot(1,2,1)
        plt.plot(fit.history['accuracy'])
        plt.plot(fit.history['val_accuracy'])
        plt.legend(['Training Accuracy', 'Validation Accuracy'])
        plt.xlabel('Epochs ')
        plt.ylabel('Accuracy')
        plt.title('Accuracy Curves')
        plt.grid(linestyle='--',alpha=0.5)

        # loss
        plt.subplot(1,2,2)
        plt.plot(fit.history['loss'])
        plt.plot(fit.history['val_loss'])
        plt.legend(['Training loss', 'Validation Loss'])
        plt.xlabel('Epochs ')
        plt.ylabel('Loss')
        plt.title('Loss Curves')
        plt.grid(linestyle='--',alpha=0.5)

        plt.savefig(graph_file)
        plt.close(fig)
        
        return fit, self.train_acc, self.test_acc

In [ ]:
# define the hyperparameters for the grid-search
from itertools import product

gridSearch = {
    "activation": ['relu', 'tanh', 'sigmoid'],
    "optimizer": ['SGD', 'RMSprop', 'Adam'],
    "lrate": [0.1, 0.01, 0.001],
    "batch_size": [16, 32, 64],
    "lreg": [0.001, 0.0001, 0.00001],
    "weight_init": ['orthogonal', 'glorot_uniform', 'he_normal'],
    "dropout": [0.2, 0.4]
}

gridSearch = list(product(*gridSearch.values()))

n_models = len(gridSearch)

In [ ]:
# run the grid-search
train_acc, test_acc = [], []

i = 0
for params in gridSearch:
    i += 1
    print('Training model ' + str(i) + ' of ' + str(n_models))
    print('Params: ' + str(params))
    nn = myDNN(params)
    _, temp_train_acc, temp_test_acc = nn.train()
    
    train_acc.append(temp_train_acc)
    test_acc.append(temp_test_acc)

In [ ]:
# Create .txt log 
log_path = 'log_book.txt'

log_text=open(log_path, "w")
log_text.write("Params, TrainAcc, TestAcc\n")
for i in range(len(train_acc)):
    log_text.write(str(gridSearch[i])+', ' + str(train_acc[i]) + ', ' + str(test_acc[i])+'\n')
log_text.close()

### 3. Rescaling

In [ ]:
# transpose and retranspose (N x L*D) data matrix
def transpose(x):
    x_transpose=[[None]*N for i in range(LD)]
    for i in range(N):
        for j in range(LD):
            x_transpose[j][i]=x[i][j]
    x_transpose=np.array(x_transpose)
    return x_transpose

def retranspose(x_transpose):
    x=[[None]*LD for i in range(N)]
    for i in range(N):
        for j in range(LD):
            x[i][j]=x_transpose[j][i]
    x=np.array(x)
    return x

In [ ]:
# import data
data = 'Randomized_Augmented_secretkeys.csv'
data = np.loadtxt(data, delimiter=',',dtype=int)
#length constants
N=len(data)
L=7
D=9
LD=L*D

#Preprocessing data
# Extract labels
labels = data[:,-1]
# Extract samples and perform one-hot encoding
samples = data[:,0].astype(str)
L = len(str(samples[0])) # number of digits for eachs string
samples = [int(i) for j in range(samples.shape[0]) for i in samples[j]] # expand string
samples = np.array(samples).reshape(-1,L) -1
samples = to_categorical(samples).reshape(-1,L*D) # one-hot encoding

In [ ]:
#RESCALING
#transpose all positions indicating the same digit in array
x=samples
x_transpose=transpose(x)

#center each digit by the mean
raw_center=[x_transpose[i]-x_transpose[i].mean() for i in range(LD)]
#rescaling not necessary since all values already in [-1,1]
x_transpose=np.array(raw_center)

#retranspose to the original shape
x_rescaled=retranspose(x_transpose)

In [ ]:
# Split samples into train and test set
Ntrain = int(0.8 * labels.shape[0])
x_train, x_test = samples[:Ntrain], samples[Ntrain:]
y_train, y_test = labels[:Ntrain], labels[Ntrain:]

x_scaled_train, x_scaled_test = x_rescaled[:Ntrain], x_rescaled[Ntrain:]

In [ ]:
#MODEL DEFINITION
#choice Hyperparameter
params=('tanh', 'SGD', 0.1, 32, 0.00001, 'glorot_uniform', 0.2)
# translator
activation = params[0]
optimizer = params[1]
lrate = params[2]
batch_size = params[3]
lreg = params[4]
winit = params[5]
dropout = params[6]

def compile_model():
    # instantiate model
    model = Sequential()
    
    model.add(Dense(inputDim, input_shape=(inputDim,), activation=activation, 
                     kernel_regularizer=l2(lreg), bias_regularizer=l2(lreg),
                    kernel_initializer=winit))
    model.add(Dropout(dropout))
    model.add(Dense(max(10,int(inputDim/2)), activation=activation, 
                         kernel_regularizer=l2(lreg), bias_regularizer=l2(lreg),
                        kernel_initializer=winit))
    model.add(Dropout(dropout))
    model.add(Dense(max(6,int(inputDim/4)), activation=activation, 
                         kernel_regularizer=l2(lreg), bias_regularizer=l2(lreg),
                        kernel_initializer=winit))
    model.add(Dropout(dropout))
    model.add(Dense(1, activation='sigmoid'))

    #optimizer
    opt = SGD(lr=lrate, decay=1e-6, momentum=0.7, nesterov=True)
        
    # compile the model
    model.compile(loss='binary_crossentropy',
                   optimizer=opt,
                   metrics=['accuracy'])
    return model

In [ ]:
#TRAINING
inputDim = L * D # samples shape

#### unscaled data ####
#model
model_unscaled=compile_model()
# train
start1=time.time()
history=model_unscaled.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=200,
          verbose=False,
          validation_data=(x_test, y_test))

#### rescaled data ####
model_rescaled=compile_model()
start2=time.time()
history1=model_rescaled.fit(x_scaled_train, y_train,
          batch_size=batch_size,
          epochs=200,
          verbose=False,
          validation_data=(x_scaled_test, y_test))
print('time unscaled', start2-start1)
print('time rescaled', time.time()-start2)

In [ ]:
#PLOT
fig = plt.figure(figsize=(15,6))
plt.subplot(1,2,1)

plt.plot(history.history['accuracy'],color='#1f77b4',label='Training Accuracy')
plt.plot(history.history['val_accuracy'],color='#2ca02c',label='Validation Accuracy')
plt.plot(history1.history['accuracy'],linestyle='--',color='#ff7f0e', label='Training Accuracy rescaled')
plt.plot(history1.history['val_accuracy'],linestyle='--',color='#d62728', label='Validation Accuracy rescaled')
plt.legend(loc='best', bbox_to_anchor=(0.5, 0.35, 0.5, 0.75),prop={'size':10})
plt.xlabel('Epochs ')
plt.ylabel('Accuracy')
plt.title('FIG 1A: Accuracy Curves')
plt.grid(linestyle='--',alpha=0.5)

plt.subplot(1,2,2)

plt.plot(history.history['loss'],color='#1f77b4',label='Training loss')
plt.plot(history.history['val_loss'],color='#2ca02c',label='Validation loss')
plt.plot(history1.history['loss'],linestyle='--',color='#ff7f0e',label='Training loss rescaled')
plt.plot(history1.history['val_loss'],linestyle='--',color='#d62728',label='validation loss rescaled')
plt.legend(loc="best",prop={'size':10})
plt.xlabel('Epochs ')
plt.ylabel('Loss')
plt.title('FIG 1B: Loss Curves')
plt.grid(linestyle='--',alpha=0.5)
plt.legend
plt.show()